In [ ]:
import geopandas as gpd
import pyvista as pv
import numpy as np

In [ ]:
gdf = gpd.read_file("./shp/dbgt_building_porch_1_lod1.shp")
gdf.head

In [ ]:
gos.estimate_utm_crs()
gos.to_crs("EPSG:32632")

In [ ]:
# read shapefile

volumes=[]
# extrude polygons one by one
for i in range(0, len(gdf.geometry.array)):
    # extract coordinates in each polygon
    pts = gpd.GeoSeries(gdf.geometry[i]).get_coordinates(include_z=True)
    # construct faces
    face=[pts.shape[0]+1] + list(range(pts.shape[0])) + [0]
    # generate polygon with coordinates and faces
    polygon=pv.PolyData(np.array(pts), faces=face)
    # generate volume by extruding polygon
    volume=polygon.extrude((0,0, gdf["UN_VOL_QE"][i]), capping=True)
    # append genereted volume to array
    volumes.append(volume)

# volumes.plot(color='white', specular=1, screenshot='extruded.png')

In [ ]:
# plot using different colors
pv.global_theme.color_cycler = "default"
pl=pv.Plotter()

for volume in volumes:
    pl.add_mesh(volume)

pl.show()